In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
#from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU, SimpleRNN
import time

In [ ]:
df = pd.read_csv('/kaggle/input/all-people-df/df')

# Ścieżka do folderu, w którym znajdują się katalogi z nagraniami osób.
file_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie nazwy podfolderów z powyższej ścieżki (są to ID nagranych osób).
subfolders = [f.name for f in os.scandir(file_path) if f.is_dir()]

# Sortuję ID nagranych osób (najpierw muszę zamienić ID na liczbę).
subfolders = sorted([int(item) for item in subfolders])
subfolders = np.array(subfolders)

df = df.loc[np.isin(np.array(df['ID']), subfolders)]

# Tworzę oddzielne ramki dla kobiet i mężczyzn.
df_woman = df[df['SEX'] == ' F '].reset_index(drop=True)
df_man = df[df['SEX'] == ' M '].reset_index(drop=True)

In [ ]:
folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

# Wyciągam wszystkie podfoldery (ID osób) z głównego folderu z nagraniami.
subfolders = [f.name for f in os.scandir(folders_path) if f.is_dir()]
# Sortuję ID osób i zamieniam je z powrotem na stringi (ID muszą być w formie tekstowej).
subfolders = sorted([int(item) for item in subfolders])
subfolders = [str(item) for item in subfolders]

# Tworzę pełne ścieżki do folderów dla każdego ID.
paths_with_ID = [folders_path + '/' + subfolder for subfolder in subfolders]

# Tworzę dwie ramki danych do przechowywania ID i sumarycznej długości nagrań dla kobiet i mężczyzn.
data_frame_for_duration_woman = pd.DataFrame(columns=['ID', 'duration'])
data_frame_for_duration_man = pd.DataFrame(columns=['ID', 'duration'])

# Pętla przez wszystkie osoby, aby obliczyć sumaryczną długość nagrań.
for path_with_ID in paths_with_ID:

    # Zbieram ścieżki do folderów wewnątrz folderu danej osoby (podfoldery).
    paths_inside_ID = [f.name for f in os.scandir(path_with_ID) if f.is_dir()]

    # Tworzę pełne ścieżki do plików nagrań (plików .flac) dla każdego folderu wewnątrz ID.
    full_paths_to_files = [path_with_ID + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie pliki audio dla danej osoby.
    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Obliczam łączną długość nagrań danej osoby.
    duration_in_seconds = 0
    for file_for_ID in all_files_for_ID:
        # Otwieram plik audio za pomocą SoundFile i obliczam długość nagrania na podstawie liczby próbek i częstotliwości próbkowania.
        with sf.SoundFile(file_for_ID) as f:
            frames = len(f)  # Liczba próbek (frames)
            sample_rate = f.samplerate  # Częstotliwość próbkowania
        duration = frames / sample_rate  # Długość nagrania w sekundach
        duration_in_seconds = duration_in_seconds + duration  # Sumowanie długości wszystkich nagrań

    # Wyciągam ID osoby z pełnej ścieżki.
    ID = path_with_ID.split('/')[-1]
    # Tworzę nowy rekord z ID i sumaryczną długością nagrań.
    new_record = [ID, duration_in_seconds]
    
    # Sprawdzam, czy ID osoby należy do kobiet i dodaję dane do odpowiedniej ramki danych.
    if np.isin(ID, df_woman['ID']):
        data_frame_for_duration_woman.loc[len(data_frame_for_duration_woman)] = new_record
    else:
        data_frame_for_duration_man.loc[len(data_frame_for_duration_man)] = new_record

    # Wyświetlam postęp pętli.
    print(ID)
    clear_output(wait=True)

# Sortuję ramki danych według długości nagrań w kolejności malejącej.
data_frame_for_duration_woman = data_frame_for_duration_woman.sort_values(by='duration', ascending=False)
data_frame_for_duration_man = data_frame_for_duration_man.sort_values(by='duration', ascending=False)

data_frame_for_duration_woman.to_csv('data_frame_for_duration_woman.csv', index=False)
data_frame_for_duration_man.to_csv('data_frame_for_duration_man.csv', index=False)

# Wybieram 50 kobiet i 50 mężczyzn o najdłuższej sumarycznej długości nagrań.
man_to_train_UBM = data_frame_for_duration_man.head(50)
woman_to_train_UBM = data_frame_for_duration_woman.head(50)

man_to_test = data_frame_for_duration_man[50:75]
woman_to_test = data_frame_for_duration_woman[50:75]

In [ ]:
with open('man_to_train_UBM.pkl', 'wb') as file:
    pickle.dump(man_to_train_UBM, file)

with open('woman_to_train_UBM.pkl', 'wb') as file:
    pickle.dump(woman_to_train_UBM, file)

with open('man_to_test.pkl', 'wb') as file:
    pickle.dump(man_to_test, file)

with open('woman_to_test.pkl', 'wb') as file:
    pickle.dump(woman_to_test, file)

In [ ]:
# Tworzę zbiory do trenowania modelu UBM i jego ewaluacji.

folders_path = '/kaggle/input/audio-to-train-model/train-clean-100'

top_50_man_paths = [folders_path + '/' + ID for ID in man_to_train_UBM['ID']]
top_50_woman_paths = [folders_path + '/' + ID for ID in woman_to_train_UBM['ID']]

data_to_train_UBM = top_50_man_paths + top_50_woman_paths

with open("data_to_train_UBM.pkl", "wb") as file:
    pickle.dump(data_to_train_UBM, file)



man_to_test = [folders_path + '/' + ID for ID in man_to_train_UBM['ID']]
woman_to_test = [folders_path + '/' + ID for ID in woman_to_train_UBM['ID']]

data_to_cross_checking = man_to_test + woman_to_test

with open("data_to_cross_checking.pkl", "wb") as file:
    pickle.dump(data_to_cross_checking, file)

In [ ]:
# Funkcja służy do dzielenia nagrań na krótkie nagrania o podanej długości i przy okazji liczy MFCC

def split_audio_to_slices(path_to_files, seconds, iterator):
    
    # Przechodzę do katalogów wewnątrz folderu osoby (ID osoby).
    # Każdy folder wewnętrzny zawiera więcej podfolderów, które mogą zawierać nagrania.
    paths_inside_ID = [f.name for f in os.scandir(path_to_files) if f.is_dir()]

    # Tworzę pełne ścieżki do podfolderów, aby przejść do wszystkich plików nagrań dla danej osoby.
    full_paths_to_files = [path_to_files + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie ścieżki do plików audio danej osoby.
    # Każdy plik powinien mieć rozszerzenie `.flac`, a wszystkie pliki są przechowywane w zmiennej `all_files_for_ID`.

    all_files_for_ID = []
    
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Łączę wszystkie nagrania danej osoby w jedno bardzo długie nagranie.
    # Używam częstotliwości próbkowania 16kHz (standardowe dla nagrań mowy).
    sr = 16000
    combined_signals = np.array([])

    for file_for_ID in all_files_for_ID:
        signal, sr = librosa.load(file_for_ID, sr=sr)
        combined_signals = np.concatenate([combined_signals, signal])



    # Długie nagranie dzielę na  fragmenty o podanej długości.
    # Fragmenty, które mają mniej niż zadeklarowane długości nagrania (resztki na końcu nagrania), są pomijane.
    list_for_parts = []
    len_of_combined_signals = len(combined_signals)
    step = seconds * sr  # Ustawienie skoku na 5 sekund
    
    for i in np.arange(start=0, stop=len_of_combined_signals-step, step=step):
        list_for_parts.append(combined_signals[i:i+step].tolist())

    parts = np.array(list_for_parts)

    

    # Liczba współczynników MFCC, które zostaną wyliczone dla każdego fragmentu nagrania (standardowe 13 współczynników).
    quantity_of_mel_coef = 40
    # Liczba filtrów melowych, które określają, ile "czapek" melowych zostanie użytych do przetwarzania sygnału.
    quantity_of_mel_filters = 60

    # Tworzę listy na współczynniki MFCC dla zbiorów uczącego i testowego.
    mfcc_list = []

    # Dla każdego fragmentu w zbiorze uczącym liczę współczynniki MFCC.
    for i in range(0, len(parts)):
        mfcc = librosa.feature.mfcc(y=parts[i], 
                                    sr=16000, 
                                    n_mfcc=quantity_of_mel_coef, 
                                    n_mels=quantity_of_mel_filters).T
        mfcc_list.append(mfcc)
        print(iterator + i/len(parts))
        clear_output(wait=True)

    mfcc_data = np.array(mfcc_list)

    # Funkcja zwraca MFCC dla nagrań o długości jednej sekundy
    return mfcc_data

In [ ]:
# Lista będzie przechowywać króciutkie nagrania, druga z nich natomiast będzie informować kto jest właścicielem nagrania.
mfcc_train_list = []
owner_of_audio = []

for i in range(0, 100):
    
    # Wywołuję funkcję split_train_test_data, aby podzielić nagrania danej osoby na zbiory treningowe i testowe.
    train_mfcc = split_audio_to_slices(data_to_train_UBM[i], 1, i)
    mfcc_train_list.extend(train_mfcc)
    
    owner_of_audio.extend([i] * len(train_mfcc))
    

In [ ]:
# Liczę pochodne MFCC aby model lepiej się nauczył

for i in range(0, len(mfcc_train_list)):
    delta = librosa.feature.delta(mfcc_train_list[i])
    delta2 = librosa.feature.delta(mfcc_train_list[i], order=2)

    one_audio = np.hstack([mfcc_train_list[i], delta, delta2])
    mfcc_train_list[i] = one_audio

    print(i/len(mfcc_train_list))
    clear_output(wait=True)

In [ ]:
with open('mfcc_train_list.pkl', 'wb') as file:
    pickle.dump(mfcc_train_list, file)

with open('owner_of_audio.pkl', 'wb') as file:
    pickle.dump(owner_of_audio, file)

In [ ]:
with open('/kaggle/input/data-to-train-rnn/data_RNN/mfcc_train_list.pkl', 'rb') as file:
    mfcc_train_list = pickle.load(file)

with open('/kaggle/input/data-to-train-rnn/data_RNN/owner_of_audio.pkl', 'rb') as file:
    owner_of_audio = pickle.load(file)

In [ ]:
# Liczę skaler aby sieć dobrze się wytrenowała

commmon_df_for_train = np.concatenate(mfcc_train_list, axis=0)

scaler = StandardScaler()

scaler.fit(commmon_df_for_train)

X_train = [scaler.transform(one_audio) for one_audio in mfcc_train_list]

X_train = np.array(X_train)
train_owner = np.array(owner_of_audio)

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [ ]:
# Wydzielam zbiór treningowy i walidacyjny do modelu
train_size = int(np.floor(len(X_train) * 0.8))

# Losowy wybór indeksów, które zostaną użyte jako dane treningowe (80% próbek).
index_of_train = np.random.choice(np.arange(0, len(X_train)), size=train_size, replace=False)

# Reszta indeksów (20% próbek będzie zbiorem walidacyjnym)
rest_of_index = ~np.isin(np.arange(0, len(X_train)), index_of_train)

X_valid = X_train[rest_of_index]
X_train = X_train[index_of_train]

valid_owner = train_owner[rest_of_index]
train_owner = train_owner[index_of_train]

In [ ]:
# Przekształcam dane treningowe (X_train), aby miały odpowiedni kształt dla sieci CNN.
# Dodaję nowy wymiar (1) na końcu, ponieważ sieci konwolucyjne oczekują wejść o formacie 4D:
# (liczba próbek, wysokość, szerokość, liczba kanałów). Tutaj mamy 1 kanał (monofoniczne nagrania).

X_train = X_train.reshape((X_train.shape[0], 
                           X_train.shape[1], 
                           X_train.shape[2], 
                           1))

# Przekształcam dane walidacyjne (X_valid) do tego samego formatu 4D co dane treningowe.
X_valid = X_valid.reshape((X_valid.shape[0], 
                           X_valid.shape[1], 
                           X_valid.shape[2], 
                           1))



# Konwertuję etykiety dla zestawu treningowego (train_owner) na postać one-hot encoding dla klasyfikacji wieloklasowej (100 klas).
# Używam funkcji to_categorical, aby zamienić numeryczne etykiety (ID osób) na macierze o rozmiarze [100], gdzie 
# każda wartość reprezentuje prawdopodobieństwo przynależności do danej klasy.
y_train = to_categorical(train_owner, num_classes=100)

# Podobnie konwertuję etykiety dla zestawu walidacyjnego na one-hot encoding.
y_valid = to_categorical(valid_owner, num_classes=100)

In [ ]:
np.save('X_train.npy', X_train)
np.save('X_valid.npy', X_valid)

np.save('y_train.npy', y_train)
np.save('y_valid.npy', y_valid)

In [15]:
X_train = np.load('/kaggle/input/train-valid-data-to-train-rnn/X_train.npy')
X_valid = np.load('/kaggle/input/train-valid-data-to-train-rnn/X_valid.npy')

y_train = np.load('/kaggle/input/train-valid-data-to-train-rnn/y_train.npy')
y_valid = np.load('/kaggle/input/train-valid-data-to-train-rnn/y_valid.npy')

In [ ]:
# Niestandardowy callback do mierzenia czasu
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []  # Lista do przechowywania czasu dla każdej epoki

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()  # Zapis początku epoki

    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time  # Czas trwania epoki
        self.times.append(epoch_time)  # Dodanie czasu do listy
        if epoch < 20:  
            print(f"Czas dla epoki {epoch + 1}: {epoch_time:.2f} sekundy")

# LSTM.

In [ ]:
# Wykrozystam stworzony callback aby zbadać który model najszybciej się uczy
time_callback = TimeHistory()

model = Sequential()

model.add(LSTM(64, input_shape=(32, 120), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(LSTM(32, return_sequences=False))
model.add(BatchNormalization())


model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(128, activation='linear', name='bottleneck'))


model.add(Dense(100, activation='softmax'))  


model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.summary()


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(
    X_train,    
    y_train,    
    epochs=200,  
    batch_size=32,  
    verbose=1,
    validation_data=(X_valid, y_valid),  
    callbacks=[reduce_lr, early_stopping, time_callback]  
)

LSTM_time = sum(time_callback.times[:20])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32, 64)         │        47,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │        33,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bottleneck (Dense)              │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 803,044 (3.06 MB)

 Trainable params: 799,268 (3.05 MB)

 Non-trainable params: 3,776 (14.75 KB)

Epoch 1/200
3774/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3793 - loss: 2.5601Czas dla epoki 1: 55.98 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 56s 13ms/step - accuracy: 0.3794 - loss: 2.5596 - val_accuracy: 0.8652 - val_loss: 0.4507 - learning_rate: 0.0010
Epoch 2/200
3773/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8134 - loss: 0.6428Czas dla epoki 2: 48.10 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - accuracy: 0.8134 - loss: 0.6428 - val_accuracy: 0.9230 - val_loss: 0.2524 - learning_rate: 0.0010
Epoch 3/200
3772/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8787 - loss: 0.4164Czas dla epoki 3: 48.93 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 49s 13ms/step - accuracy: 0.8787 - loss: 0.4164 - val_accuracy: 0.9433 - val_loss: 0.1873 - learning_rate: 0.0010
Epoch 4/200
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9026 - loss: 0.3323Czas dla epoki 4: 48.46 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - accuracy: 0.9026 - loss: 0.3323 - 

In [18]:
model.save('model_LSTM.h5')
np.save('LSTM_time.npy', LSTM_time)

# GRU.

In [ ]:
time_callback = TimeHistory()


model = Sequential()



model.add(GRU(64, input_shape=(32, 120), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(GRU(32, return_sequences=False))
model.add(BatchNormalization())



model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))  

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(128, activation='linear', name = 'bottleneck'))


model.add(Dense(100, activation='softmax'))  


model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.summary()


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train,    
    y_train,    
    epochs=200,  
    batch_size=32, 
    verbose = 1,
    validation_data=(X_valid, y_valid),  
    callbacks=[reduce_lr, early_stopping, time_callback]  
)

GRU_time = sum(time_callback.times[:20])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 32, 64)         │        35,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │        33,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bottleneck (Dense)              │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 788,388 (3.01 MB)

 Trainable params: 784,612 (2.99 MB)

 Non-trainable params: 3,776 (14.75 KB)

Epoch 1/200
3771/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4096 - loss: 2.4371Czas dla epoki 1: 56.62 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 57s 14ms/step - accuracy: 0.4098 - loss: 2.4358 - val_accuracy: 0.9125 - val_loss: 0.2878 - learning_rate: 0.0010
Epoch 2/200
3774/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8336 - loss: 0.5547Czas dla epoki 2: 51.67 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 52s 14ms/step - accuracy: 0.8336 - loss: 0.5547 - val_accuracy: 0.9393 - val_loss: 0.1966 - learning_rate: 0.0010
Epoch 3/200
3773/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8790 - loss: 0.4055Czas dla epoki 3: 51.25 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 51s 14ms/step - accuracy: 0.8790 - loss: 0.4055 - val_accuracy: 0.9572 - val_loss: 0.1422 - learning_rate: 0.0010
Epoch 4/200
3774/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9004 - loss: 0.3307Czas dla epoki 4: 51.74 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 52s 14ms/step - accuracy: 0.9004 - loss: 0.3307 - 

In [20]:
model.save('model_GRU.h5')
np.save('GRU_time.npy', GRU_time)

# Klasyczna RNN.

In [ ]:
time_callback = TimeHistory()

model = Sequential()


model.add(SimpleRNN(64, input_shape=(32, 120), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(SimpleRNN(32, return_sequences=False))
model.add(BatchNormalization())



model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))    

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(128, activation='linear', name = 'bottleneck'))

model.add(Dense(100, activation='softmax'))  


model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.summary()


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train,    
    y_train,    
    epochs=200,  
    batch_size=32, 
    verbose = 1,
    validation_data=(X_valid, y_valid),  
    callbacks=[reduce_lr, early_stopping, time_callback]  
)

RNN_time = sum(time_callback.times[:20])

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32, 64)         │        11,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 32, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1024)           │        33,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bottleneck (Dense)              │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 758,212 (2.89 MB)

 Trainable params: 754,436 (2.88 MB)

 Non-trainable params: 3,776 (14.75 KB)

Epoch 1/200


I0000 00:00:1732115749.829691      64 service.cc:145] XLA service 0x5cdef92da500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732115749.829754      64 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  22/3775 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - accuracy: 0.0090 - loss: 5.5244

I0000 00:00:1732115758.700762      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3775/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1039 - loss: 3.9252Czas dla epoki 1: 53.97 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 54s 11ms/step - accuracy: 0.1039 - loss: 3.9251 - val_accuracy: 0.2446 - val_loss: 2.9358 - learning_rate: 0.0010
Epoch 2/200
3769/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2292 - loss: 2.9865Czas dla epoki 2: 31.93 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 32s 8ms/step - accuracy: 0.2292 - loss: 2.9865 - val_accuracy: 0.2811 - val_loss: 2.6960 - learning_rate: 0.0010
Epoch 3/200
3769/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2376 - loss: 2.9466Czas dla epoki 3: 31.52 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 32s 8ms/step - accuracy: 0.2376 - loss: 2.9467 - val_accuracy: 0.2718 - val_loss: 2.7486 - learning_rate: 0.0010
Epoch 4/200
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2211 - loss: 3.0219Czas dla epoki 4: 32.18 sekundy
3775/3775 ━━━━━━━━━━━━━━━━━━━━ 32s 9ms/step - accuracy: 0.2211 - loss: 3.0219 - val_accuracy: 0.28

In [22]:
model.save('model_RNN.h5')
np.save('RNN_time.npy', RNN_time)